In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u

In [2]:
t = pd.read_csv("ds9.tsv", delimiter="\t")

In [3]:
t

,_RAJ2000,_DEJ2000,GSC2.3,RAJ2000,DEJ2000,Epoch,Fmag,jmag,Vmag,Nmag,Class,Status,a,e
0,250.228741,36.322915,N61Q000270,250.228741,36.322915,2000.000,NaN,NaN,10.73,NaN,0,99999900,NaN,NaN
1,250.595559,36.345197,N61Q000262,250.595559,36.345197,2000.000,NaN,NaN,10.86,NaN,0,99999900,NaN,NaN
2,250.248615,36.467774,N61I000452,250.248615,36.467774,2000.000,NaN,NaN,10.87,NaN,0,99999900,NaN,NaN
3,250.212156,36.452658,N61I000462,250.212156,36.452658,2000.000,NaN,NaN,10.99,NaN,0,99999900,NaN,NaN
4,250.160369,36.385155,N61I000505,250.160369,36.385155,2000.000,NaN,NaN,11.14,NaN,0,99999900,NaN,NaN
5,250.203992,36.316397,N61Q000273,250.203992,36.316397,1993.322,10.47,12.71,11.31,9.19,0,101111207,11.32,0.02
6,250.634790,36.510696,N61D000054,250.634790,36.510696,2000.000,NaN,NaN,11.41,NaN,0,99999900,NaN,NaN
7,250.331587,36.354406,N61Q000253,250.331587,36.354406,1993.322,11.03,12.74,11.65,10.46,0,1101111507,10.22,0.11
8,250.032350,36.373635,N61I000512,250.032350,36.373635,1991.570,NaN,NaN,11.71,NaN,0,99999900,NaN,NaN
9,250.653024,36.626659,N61D000040,250.653024,36.626659,2000.000,NaN,NaN,11.82,NaN,0,99999900,NaN,NaN


In [4]:
star_0 = SkyCoord(t.RAJ2000[0], t.DEJ2000[0], unit="deg")
star_1 = SkyCoord(t.RAJ2000[1], t.DEJ2000[1], unit="deg")
star_2 = SkyCoord(t.RAJ2000[2], t.DEJ2000[2], unit="deg")
star_3 = SkyCoord(t.RAJ2000[3], t.DEJ2000[3], unit="deg")
star_4 = SkyCoord(t.RAJ2000[4], t.DEJ2000[4], unit="deg")
star_5 = SkyCoord(t.RAJ2000[5], t.DEJ2000[5], unit="deg")

In [5]:
dist01 = star_0.separation(star_1)
dist23 = star_2.separation(star_3)
dist45 = star_4.separation(star_5)

In [6]:
dist01, dist23, dist45

(<Angle 0.29633858 deg>, <Angle 0.03298976 deg>, <Angle 0.07721443 deg>)

In [7]:
ACO_ar = np.array([[1632.4678, 1915.1682],
                   [3289.5154, 1918.6404],
                   [1782.9244, 2714.4204],
                   [1611.7318, 2643.4356],
                   [1351.6156, 2283.594],
                   [1515.1888, 1893.2748]])

t_ACO = pd.DataFrame(ACO_ar, columns=["x", "y"])

In [8]:
t_ACO

,x,y
0,1632.4678,1915.1682
1,3289.5154,1918.6404
2,1782.9244,2714.4204
3,1611.7318,2643.4356
4,1351.6156,2283.5940
5,1515.1888,1893.2748


In [9]:
def calc_dist(table, i, j):
    return ((table.at[i,"x"]-table.at[j,"x"])**2+(table.at[i,"y"]-table.at[j,"y"])**2)**(1/2)

In [10]:
ACO01 = calc_dist(t_ACO, 0, 1)
ACO23 = calc_dist(t_ACO, 2, 3)
ACO45 = calc_dist(t_ACO, 4, 5)

In [11]:
ACO01, ACO23, ACO45

(np.float64(1657.0512378434776),
 np.float64(185.32605894962546),
 np.float64(423.20830526689815))

In [12]:
t_final = pd.DataFrame(t[["GSC2.3", "RAJ2000", "DEJ2000"]][:6])

In [13]:
t_final[["x","y"]] = t_ACO

In [14]:
t_final["sep_pixel"]=[ACO01, np.nan, ACO23, np.nan, ACO45, np.nan]
t_final["sep_arcsec"]=[dist01.to(u.arcsec).value, np.nan, dist23.to(u.arcsec).value, np.nan, dist45.to(u.arcsec).value, np.nan]

In [15]:
t_final["pixel_scale"] = t_final["sep_arcsec"]/t_final["sep_pixel"]

In [16]:
t_final

,GSC2.3,RAJ2000,DEJ2000,x,y,sep_pixel,sep_arcsec,pixel_scale
0,N61Q000270,250.228741,36.322915,1632.4678,1915.1682,1657.051238,1066.818883,0.643806
1,N61Q000262,250.595559,36.345197,3289.5154,1918.6404,NaN,NaN,NaN
2,N61I000452,250.248615,36.467774,1782.9244,2714.4204,185.326059,118.763125,0.640833
3,N61I000462,250.212156,36.452658,1611.7318,2643.4356,NaN,NaN,NaN
4,N61I000505,250.160369,36.385155,1351.6156,2283.5940,423.208305,277.971938,0.656821
5,N61Q000273,250.203992,36.316397,1515.1888,1893.2748,NaN,NaN,NaN


In [17]:
avg = np.sum(t_final.pixel_scale)/3
err = np.abs((avg-0.643)/0.643)*100
print(f"Average pixel scale is {avg:.4f}\"/pixel and % error is {err:.4f}")

Average pixel scale is 0.6472"/pixel and % error is 0.6459
